<a href="https://colab.research.google.com/github/rajshekharM/tvm/blob/master/Copy_of_02_TVM_Tutorial_Relay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uwsampl/tutorial/blob/master/notebook/02_TVM_Tutorial_Relay.ipynb)

Please run the following block to ensure TVM is setup for *this notebook*, each notebook may have its own runtime.

In [0]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    ! gsutil cp "gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz" /tmp/tvm.tar.gz
    ! mkdir -p /tvm
    ! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
    ! ls -la /tvm
    ! bash /tvm/package.sh
    ! pip install onnx 
    ! sudo apt install clang-format
    # ! pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
    # ! pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl
    # Add TVM to the Python path.
    import sys
    import os
    sys.path.append('/tvm/python')
    sys.path.append('/tvm/topi/python')
    sys.path.append('/tvm/vta/python')
    os.environ['TVM_HOME'] = '/tvm'
else:
    print("Notebook executing locally, skipping Colab setup ...")

Copying gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz...
/ [1 files][119.5 MiB/119.5 MiB]                                                
Operation completed over 1 objects/119.5 MiB.                                    
total 164
drwxr-xr-x 21 root root  4096 Nov  8 20:17 .
drwxr-xr-x  1 root root  4096 Nov  8 20:16 ..
drwx------  8 root root  4096 May 31 08:14 3rdparty
drwx------ 12 root root  4096 May 31 08:14 apps
drwx------  3 root root  4096 Jun 19 07:58 build
drwx------  4 root root  4096 May 31 08:14 cmake
-rw-------  1 root root 11053 Jun 19 04:54 CMakeLists.txt
drwx------  6 root root  4096 May 31 08:14 conda
-rw-------  1 root root  5736 Jun 19 04:54 CONTRIBUTORS.md
drwx------  3 root root  4096 May 31 08:14 docker
drwx------ 11 root root  4096 May 31 08:14 docs
drwx------  4 root root  4096 May 31 08:14 golang
drwx------  3 root root  4096 May 31 08:14 include
-rw-------  1 root root 10607 Jun 19 04:54 Jenkinsfile
drwx------  6 root root  4096 May 31 

In [0]:
if IN_COLAB:
    ! cd /; git clone https://github.com/uwsampl/relay-aot
    sys.path.append('/relay-aot')

Cloning into 'relay-aot'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 330 (delta 51), reused 74 (delta 39), pack-reused 236
Receiving objects: 100% (330/330), 71.37 KiB | 3.40 MiB/s, done.
Resolving deltas: 100% (177/177), done.


# Relay: an Extensible Deep Learning IR

Last year TVM introduced Relay IR – a second generation high-level IR for deep learning. 

Relay's design comes from a simple insight that the critical difference between regular IRs
and deep learning IRs are the primitive values they manipulate. Relay is designed using well
known insights from the programming languages community coupled with TVM's existing 
infrastructure to provide state of the art performance. 

If you are familiar with ideas from programming languages or existing computation graph
representations, we will connect Relay to your existing knowledge during this tutorial.

We will first cover the design of Relay then elaborate on how one can use it to 
accomplish a wide variety of tasks. This portion of the tutorial is focused  
on Relay, but Relay will be discussed throughout the day day. Relay serves
as the interface layer to TVM.

In [0]:
import tvm
from tvm import relay
import tvm.relay.testing
from tvm.relay.expr_functor import ExprMutator, ExprVisitor
import torch
import torchvision
import onnx
import numpy
import aot
import logging

logging.disable(logging.WARNING)

## Language 

We will first briefly introduce the concepts of Relay below.
You can find a full language specification [here](https://docs.tvm.ai/langref/index.html).

### Variables 

In [0]:
# A single Relay variable, the string is just a hint
x = relay.var('x')

# A Relay variable with a different dtype, defaults to float32.
x = relay.var('x', dtype='int32')

# A Relay variable with a different shape.
x = relay.var('x', shape=(10, 1))

### Operators

Relay provides high performance operators defined in TVM that implement the primitive operations needed by deep learning applications. Operators can be applied to arguments just like regular Python or C++ functions. Common arithemetic operations are provided both via names and operator overloading.

Variables can be used to construct Relay *expressions* which replace the concept of graphs present in previous frameworks. A Relay expression can be viewed much like a graph with extra functionality as we will see as we go
forward.

In [0]:
w = relay.op.add(x, x)
print(w)

v0.0.1
free_var %x: Tensor[(10, 1), float32]
add(%x, %x)


In [0]:
z = x + x
print(z)

v0.0.1
free_var %x: Tensor[(10, 1), float32]
add(%x, %x)


### Functions

The fundamental packaging of computation in Relay is the function. A function is a combination of a set of inputs
and a Relay expression. Relay functions are no different than ones in programming languages today. They replace named subgraphs.

In [0]:
f = relay.Function([x], z)
print(f)

v0.0.1
fn (%x: Tensor[(10, 1), float32]) {
  add(%x, %x)
}


### Module

Finally, we can give functions a global name and package many of them together into a module. When we add a function to the module, it will be type checked before hand.

When we print the module, you can see the program annotated with all type information. 

In [0]:
mod = relay.Module({})
fname = relay.GlobalVar('f')
mod[fname] = f

print(mod)

v0.0.1
def @f(%x: Tensor[(10, 1), float32]) -> Tensor[(10, 1), float32] {
  add(%x, %x) /* ty=Tensor[(10, 1), float32] */
}



## Frontends

Relay comes with a variety of frontends and supports most major frameworks, including TensorFlow, PyTorch, MxNet, ONNX, Keras and Caffe2.

Below we provide a couple examples of using these frontends to import models into Relay.

You can find specific tutorials on deploying pretrained models below:  

- [ONNX](https://docs.tvm.ai/tutorials/frontend/from_onnx.html#sphx-glr-tutorials-frontend-from-onnx-py)
- [TensorFlow](https://docs.tvm.ai/tutorials/frontend/from_tensorflow.html#sphx-glr-tutorials-frontend-from-tensorflow-py)
- [Keras](https://docs.tvm.ai/tutorials/frontend/from_keras.html#sphx-glr-tutorials-frontend-from-keras-py)
- [PyTorch](https://tvm.ai/2019/05/30/pytorch-frontend)
- [Caffe2](https://docs.tvm.ai/tutorials/frontend/from_caffe2.html#sphx-glr-tutorials-frontend-from-caffe2-py)

In [0]:
torch_resnet18 = torchvision.models.resnet18()
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(torch_resnet18, dummy_input, "resnet.onnx",  export_params=True, verbose=True)

graph(%0 : Float(1, 3, 224, 224),
      %conv1.weight : Float(64, 3, 7, 7),
      %bn1.weight : Float(64),
      %bn1.bias : Float(64),
      %bn1.running_mean : Float(64),
      %bn1.running_var : Float(64),
      %bn1.num_batches_tracked : Long(),
      %layer1.0.conv1.weight : Float(64, 64, 3, 3),
      %layer1.0.bn1.weight : Float(64),
      %layer1.0.bn1.bias : Float(64),
      %layer1.0.bn1.running_mean : Float(64),
      %layer1.0.bn1.running_var : Float(64),
      %layer1.0.bn1.num_batches_tracked : Long(),
      %layer1.0.conv2.weight : Float(64, 64, 3, 3),
      %layer1.0.bn2.weight : Float(64),
      %layer1.0.bn2.bias : Float(64),
      %layer1.0.bn2.running_mean : Float(64),
      %layer1.0.bn2.running_var : Float(64),
      %layer1.0.bn2.num_batches_tracked : Long(),
      %layer1.1.conv1.weight : Float(64, 64, 3, 3),
      %layer1.1.bn1.weight : Float(64),
      %layer1.1.bn1.bias : Float(64),
      %layer1.1.bn1.running_mean : Float(64),
      %layer1.1.bn1.running_var 

In [0]:
onnx_resnet18 = onnx.load('resnet.onnx')
func, params = relay.frontend.from_onnx(onnx_resnet18, shape={ '0': (1, 3, 224, 224) })
print(func)

v0.0.1
def @main(%v0: Tensor[(1, 3, 224, 224), float32], %conv1.weight: Tensor[(64, 3, 7, 7), float32], %bn1.weight: Tensor[(64,), float32], %bn1.bias: Tensor[(64,), float32], %bn1.running_mean: Tensor[(64,), float32], %bn1.running_var: Tensor[(64,), float32], %layer1.0.conv1.weight: Tensor[(64, 64, 3, 3), float32], %layer1.0.bn1.weight: Tensor[(64,), float32], %layer1.0.bn1.bias: Tensor[(64,), float32], %layer1.0.bn1.running_mean: Tensor[(64,), float32], %layer1.0.bn1.running_var: Tensor[(64,), float32], %layer1.0.conv2.weight: Tensor[(64, 64, 3, 3), float32], %layer1.0.bn2.weight: Tensor[(64,), float32], %layer1.0.bn2.bias: Tensor[(64,), float32], %layer1.0.bn2.running_mean: Tensor[(64,), float32], %layer1.0.bn2.running_var: Tensor[(64,), float32], %layer1.1.conv1.weight: Tensor[(64, 64, 3, 3), float32], %layer1.1.bn1.weight: Tensor[(64,), float32], %layer1.1.bn1.bias: Tensor[(64,), float32], %layer1.1.bn1.running_mean: Tensor[(64,), float32], %layer1.1.bn1.running_var: Tensor[(64,),

## Text Format

Relay has a textual representation that can be used to write and print programs. The textual format is still being stablized but can still be of great use today. For example, instead of providing inscrutable graph representations of programs, we can produce human readable output by default.

There are a few different ways to interact with the textual format. The first is to just print out a Relay expression as we have seen above.

In [0]:
mlp, params = relay.testing.mlp.get_workload(1)
print(mlp)

v0.0.1
fn (%data: Tensor[(1, 1, 28, 28), float32], %fc1_weight: Tensor[(128, 784), float32], %fc1_bias: Tensor[(128,), float32], %fc2_weight: Tensor[(64, 128), float32], %fc2_bias: Tensor[(64,), float32], %fc3_weight: Tensor[(10, 64), float32], %fc3_bias: Tensor[(10,), float32]) -> Tensor[(1, 10), float32] {
  %0 = nn.batch_flatten(%data) /* ty=Tensor[(1, 784), float32] */
  %1 = nn.dense(%0, %fc1_weight, units=128) /* ty=Tensor[(1, 128), float32] */
  %2 = nn.bias_add(%1, %fc1_bias, axis=-1) /* ty=Tensor[(1, 128), float32] */
  %3 = nn.relu(%2) /* ty=Tensor[(1, 128), float32] */
  %4 = nn.dense(%3, %fc2_weight, units=64) /* ty=Tensor[(1, 64), float32] */
  %5 = nn.bias_add(%4, %fc2_bias, axis=-1) /* ty=Tensor[(1, 64), float32] */
  %6 = nn.relu(%5) /* ty=Tensor[(1, 64), float32] */
  %7 = nn.dense(%6, %fc3_weight, units=10) /* ty=Tensor[(1, 10), float32] */
  %8 = nn.bias_add(%7, %fc3_bias, axis=-1) /* ty=Tensor[(1, 10), float32] */
  nn.softmax(%8) /* ty=Tensor[(1, 10), float32] */
}

By default the pretty printer renders the code without metadata. The pretty printer omits metadata because the metadata, which contains information such as constants, is often unreadable. For instance, imagine we perform an optimization, such as inlining the parameters into the program for further optimization. The metadata would include 100s of megabytes of parameters.

In [0]:
def inline_parameters(expr, params):
    param_map = dict((p.name_hint, p) for p in expr.params)
    params = dict((param_map[k], relay.const(params[k])) for k in params)
    new_body = relay.bind(expr.body, params)
    return relay.Function(relay.ir_pass.free_vars(new_body), new_body)

inline_mlp = inline_parameters(mlp, params)
print(inline_mlp.astext(show_meta_data=True))

v0.0.1
fn (%data: Tensor[(1, 1, 28, 28), float32]) {
  %0 = nn.batch_flatten(%data) /* ty=Tensor[(1, 784), float32] */
  %1 = nn.dense(%0, meta[relay.Constant][0], units=128)
  %2 = nn.bias_add(%1, meta[relay.Constant][1], axis=-1)
  %3 = nn.relu(%2)
  %4 = nn.dense(%3, meta[relay.Constant][2], units=64)
  %5 = nn.bias_add(%4, meta[relay.Constant][3], axis=-1)
  %6 = nn.relu(%5)
  %7 = nn.dense(%6, meta[relay.Constant][4], units=10)
  %8 = nn.bias_add(%7, meta[relay.Constant][5], axis=-1)
  nn.softmax(%8)
}
/* meta data */
{
  "root": 1, 
  "nodes": [
    {
      "type_key": ""
    }, 
    {
      "type_key": "StrMap", 
      "keys": [
        "relay.Constant"
      ], 
      "data": [2]
    }, 
    {
      "type_key": "Array", 
      "data": [3, 4, 5, 6, 7, 8]
    }, 
    {
      "type_key": "relay.Constant", 
      "attrs": {
        "_checked_type_": "0", 
        "data": "0", 
        "span": "0"
      }
    }, 
    {
      "type_key": "relay.Constant", 
      "attrs": {
        "_

Relay's pretty printer also allows users to attach debugging output and metadata to the IR. For example you can see the type information on the example above, but we can also customize the annotation process by passing a callback for annotating nodes. 

In [0]:
i = 0 
def ann(*args):
    global i
    i += 1
    return f" <expression: {i}>"

print(mlp.astext(show_meta_data=True, annotate=ann))

v0.0.1
fn (%data: Tensor[(1, 1, 28, 28), float32], %fc1_weight: Tensor[(128, 784), float32], %fc1_bias: Tensor[(128,), float32], %fc2_weight: Tensor[(64, 128), float32], %fc2_bias: Tensor[(64,), float32], %fc3_weight: Tensor[(10, 64), float32], %fc3_bias: Tensor[(10,), float32]) -> Tensor[(1, 10), float32] {
  %0 = nn.batch_flatten <expression: 1>(%data) <expression: 2>
  %1 = nn.dense <expression: 3>(%0, %fc1_weight, units=128) <expression: 4>
  %2 = nn.bias_add <expression: 5>(%1, %fc1_bias, axis=-1) <expression: 6>
  %3 = nn.relu <expression: 7>(%2) <expression: 8>
  %4 = nn.dense <expression: 3>(%3, %fc2_weight, units=64) <expression: 9>
  %5 = nn.bias_add <expression: 5>(%4, %fc2_bias, axis=-1) <expression: 10>
  %6 = nn.relu <expression: 7>(%5) <expression: 11>
  %7 = nn.dense <expression: 3>(%6, %fc3_weight, units=10) <expression: 12>
  %8 = nn.bias_add <expression: 5>(%7, %fc3_bias, axis=-1) <expression: 13>
  nn.softmax <expression: 14>(%8) <expression: 15>
} <expression: 16>


Finally, an important part of the Relay text format is the ability to load Relay code 
like a normal programming language. We can use the Relay parser to parse code. We actually do this to define the Relay
*prelude* the small standard library of utilities shipped in Relay. 

## Executing Relay

Now that we have looked at how to write and manipulate a Relay program, we will show you how to run one. Relay has multiple execution mechanisms: a custom *debug interpreter* for Relay which can be used for experimentation and debugging, TVM's older graph runtime, the existing execution mechanism, and Relay VM. Relay VM is a newly designed execution mechanism with the goal to smoothly execute all of Relay efficiently. 

We provide a high level interface which imposes some wrapping overhead but enables quick experimentation with each API. 

In [0]:
mod = relay.Module()
debug_ex = relay.create_executor('debug', mod=mod)
graph_ex = relay.create_executor('graph', mod=mod)
vm_ex = relay.create_executor('vm', mod=mod)

Each executor can be used to evaluate an expression given a Relay module, in this case we use an empty module, and will just evaluate the same expression, a MLP, using each one.

In [0]:
debug_mlp = debug_ex.evaluate(mlp)
graph_mlp = graph_ex.evaluate(mlp)
vm_mlp = vm_ex.evaluate(mlp)

...100%, 0.02 MB, 129 KB/s, 0 seconds passed


Each one can be called like a normal Python function with the inputs passed as positional arguments and the parameters as keyword arguments.

In [0]:
data = numpy.random.rand(1, 1, 28, 28).astype('float32')
print("Debug: ", debug_mlp(data, **params))
print("Graph: ", graph_mlp(data, **params))
print("VM: ", vm_mlp(data, **params))

Debug:  [[0.10704241 0.18922117 0.04617432 0.05432151 0.08334016 0.24642973
  0.04587974 0.02491589 0.1437427  0.05893242]]
Graph:  [[0.10704241 0.18922117 0.04617432 0.05432151 0.08334016 0.24642973
  0.04587974 0.02491589 0.1437427  0.05893242]]
VM:  [[0.10704241 0.18922117 0.04617432 0.05432151 0.08334016 0.24642973
  0.04587974 0.02491589 0.1437427  0.05893242]]


### Virtual Machine

The Relay virtual machine is worth highlighting, it is a brand new runtime mechanism for Relay which is beginning to stablize. We encourage new users to check it out and provide feedback on its design, perforomance, and more. The VM enables support for non-standard aspects of Relay including control-flow, closures, and data structures.

For more details on the virtual machine see [here](https://github.com/dmlc/tvm/issues/2810).

## Optimizations

Defining optimizations to transform your program is straight forward and easy to do in Relay.

For example let's define a simple constant evaluator for Relay. We will first begin by defining a simpe visitor which checks whether a Relay operator invocation is "constant". We will do this by assuming any expression we see is constant, and then marking certain expressions as non-constant.

Operators and constants are always constant, but a Relay call can only be constant if its arguments and functions are also constant. 

We will modify the visit function to mark anythign else as non-constant, and then visit the expression recursively
applying these rules.

In [0]:
class IsConstant(ExprVisitor):
    def __init__(self):
        self.is_const = True
        super().__init__()
    
    def visit_const(self, const):
        self.is_const = self.is_const and True
    
    def visit_op(self, op):
        self.is_const = self.is_const and True
        
    def visit_call(self, call):
        for arg in call.args:
            self.visit(arg)
        
        self.visit(call.op)
        
    def visit(self, expr):
        if not isinstance(expr, (relay.Op, relay.Call, relay.Constant)):
            self.is_const = False
        
        return super().visit(expr)


We can then wrap this in a helper function which applies the visitor and returns the result, we will use this predicate below. 

In [0]:
    
def is_const(expr):
  isc = IsConstant()
  isc.visit(expr)
  return isc.is_const

With this pass in hand we can define a simple contant folder which removes constant operator invocations. 

In [0]:
class ConstantFolder(ExprMutator):
    def visit_call(self, call):
        if is_const(call):
            ex = relay.create_executor('debug')
            cexpr = ex.evaluate(call)
            return relay.const(cexpr.asnumpy())
        else:
            return super().visit_call(call)

def const_fold(expr):
    return ConstantFolder().visit(expr)

In [0]:
c1 = relay.const(numpy.ones((10, 1), 'float32'))
c2 = relay.const(numpy.ones((1, 1), 'float32'))
e = c1 + c2
print("Input Expr: ", e)
const = const_fold(e)
assert isinstance(const, relay.Constant)
print("Output: ", const.data)

Input Expr:  v0.0.1
add(meta[relay.Constant][0], meta[relay.Constant][1])
// meta data omitted. you can use show_meta_data=True to include meta data
Output:  [[2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]]


## Pass Manager

Relay has a flexible and configurable pass manager with an elegant API that be used to easily compose and schedule pass pipelines. We believe an easy-to-configure pipeline is important to enable intelligent exploration of a variety of optimizations, orders, and optimization parameters.

In [0]:
resnet, params = relay.testing.resnet.get_workload()

The Relay pass infrastructure takes inspiration from both traditional designs from the compiler world as well as APIs from deep learning. In particular our pass manager uses a uniform interface similar to the `Layer` abstraction provided by many deep learning frameworks. Our passes are defined as parametric functions from Module to Module.

For example we define another type of pass, `Sequential` which runs a set of passes serially over a module. Below we construct a compiler pipeline from a set of transformations, we will first type check, perform cosntant folding, eliminate common subexpressions, and the perform layout transformations.

In [0]:
seq = relay.transform.Sequential([
    relay.transform.InferType(),
    relay.transform.SimplifyInference(),
    relay.transform.FoldConstant(),
    relay.transform.EliminateCommonSubexpr(),
    relay.transform.AlterOpLayout()
])

Finally we will construct a module using the ResNet as our *main* function. To do so we:

- Use `relay.build_config` to setup the scoped build config, these scoped configs can be used with Python's `with` statement. After the block ends the configuration will be reset to the defaults.
- Use `tvm.target.create` in a scoped fashion as well to set the target to `llvm`.
- We can now optimize the module with this information and get it out.

In [0]:
mod = relay.Module({"main": resnet})
with relay.build_config(opt_level=3):
    with tvm.target.create("llvm"):
        mod = seq(mod)
        print(mod["main"])
        

v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32], %bn_data_gamma: Tensor[(3,), float32], %bn_data_beta: Tensor[(3,), float32], %bn_data_moving_mean: Tensor[(3,), float32], %bn_data_moving_var: Tensor[(3,), float32], %conv0_weight: Tensor[(64, 3, 7, 7), float32], %bn0_gamma: Tensor[(64,), float32], %bn0_beta: Tensor[(64,), float32], %bn0_moving_mean: Tensor[(64,), float32], %bn0_moving_var: Tensor[(64,), float32], %stage1_unit1_bn1_gamma: Tensor[(64,), float32], %stage1_unit1_bn1_beta: Tensor[(64,), float32], %stage1_unit1_bn1_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn1_moving_var: Tensor[(64,), float32], %stage1_unit1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_bn2_gamma: Tensor[(64,), float32], %stage1_unit1_bn2_beta: Tensor[(64,), float32], %stage1_unit1_bn2_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn2_moving_var: Tensor[(64,), float32], %stage1_unit1_conv2_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_sc_weight: Tensor[(64, 64, 1, 1)

We can easily lift functions operating on expressions to be passes. We simply use `relay.transform.function_pass`.

For example we can lift our new constant folder to be a pass in the pass manager with a few extra lines.

In [0]:
@relay.transform.function_pass(opt_level=1, name="MyFoldConstant")
def MyFoldConstant(func, mod, ctx):
  return const_fold(func)

For example we can now repeat the same pipeline as above, with our new pass.

In [0]:
seq = relay.transform.Sequential([
    relay.transform.InferType(),
    relay.transform.SimplifyInference(),
    MyFoldConstant,
    relay.transform.EliminateCommonSubexpr(),
    relay.transform.AlterOpLayout()
])

mod = relay.Module({"main": resnet})
with relay.build_config(opt_level=3):
    with tvm.target.create("llvm"):
        mod = seq(mod)

## Quantization

Relay supports an automatic quantization framework which can be used to implement a variety of quantization schemes. You can find more details about it in our recent paper as well as it being used in action here.

Below we will apply the default quantization scheme to ResNet-18.

Quantization proceeds in a few steps, first we start with a raw program that has yet been quantized. 

Our quantization pass determines where we can apply quantization, for example at convolutions.

We begin with the unmodified program for example a convolution pipeline with batch norm and relu.

![Unmodified Program](https://www.dropbox.com/s/1w7nb845fmmbcof/Screenshot%202019-06-21%2021.33.17.png?raw=1)

We then insert simualted quantized operations with simulate the effect of quantization parametrized by the number of bits, the range of the value and the sign parameter.

![Simulated Quantized Program](https://www.dropbox.com/s/rfy8i7u5uahpacn/Screenshot%202019-06-21%2021.33.24.png?raw=1)

After this we perform calibration a process by which we select values for these parameters and then we fold the quantization directly into the operators producing quantized operators which now operate in the target bitwidth and domain, for example `int8` and `int32`.

![Final Program](https://www.dropbox.com/s/9z263hrwp1p6ra8/Screenshot%202019-06-21%2021.33.30.png?raw=1)







In [0]:
resnet, params = relay.testing.resnet.get_workload()

with relay.quantize.qconfig():
    qresnet = relay.quantize.quantize(resnet, params)
    print(qresnet)

v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32]) -> Tensor[(1, 1000), float32] {
  %0 = add(%data, meta[relay.Constant][0] /* ty=Tensor[(3, 1, 1), float32] */ /* ty=Tensor[(3, 1, 1), float32] */) /* ty=Tensor[(1, 3, 224, 224), float32] */
  %1 = nn.conv2d(%0, meta[relay.Constant][1] /* ty=Tensor[(64, 3, 7, 7), float32] */ /* ty=Tensor[(64, 3, 7, 7), float32] */, strides=[2, 2], padding=[3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */
  %2 = add(%1, meta[relay.Constant][2] /* ty=Tensor[(64, 1, 1), float32] */ /* ty=Tensor[(64, 1, 1), float32] */) /* ty=Tensor[(1, 64, 112, 112), float32] */
  %3 = nn.relu(%2) /* ty=Tensor[(1, 64, 112, 112), float32] */
  %4 = nn.max_pool2d(%3, pool_size=[3, 3], strides=[2, 2], padding=[1, 1]) /* ty=Tensor[(1, 64, 56, 56), float32] */
  %5 = add(%4, meta[relay.Constant][3] /* ty=Tensor[(64, 1, 1), float32] */ /* ty=Tensor[(64, 1, 1), float32] */) /* ty=Tensor[(1, 64, 56, 56), float32] */
  %6 = nn.relu(%5) /* ty=Ten

## Heterogeneous Execution

Relay supports a high-level interface for scheduling computation across multiple heterogeneous devices. An interesting property of this pass is that it is not special. It is built using Relay's standard machinery for
passes. 

We implement this by using an annotation to mark which computations we would like to schedule on each device, 
and a pass inserts all the appropriate calls to synchronize memory across devices. 

The below pass uses this machinery to schedule all convolutions onto the GPU.

In [0]:
class ScheduleConv2d(ExprMutator):
    def __init__(self, device):
        self.device = device
        super().__init__()

    def visit_call(self, expr):
        visit = super().visit_call(expr)
        if expr.op == tvm.relay.op.get("nn.conv2d"):
            return relay.annotation.on_device(visit, self.device)
        else:
            return visit

def schedule_conv2d_on_gpu(expr):
    sched = ScheduleConv2d(tvm.gpu(0))
    return sched.visit(expr)

We can grab a model, we provide a few basic models in Relay's testing library. By default when printing a model we will see it rendered in Relay's textual format.

In [0]:
# We can grab a model, we provide a few basic models in Relay's testing library.
resnet, params = relay.testing.resnet.get_workload()
print(resnet)

v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32], %bn_data_gamma: Tensor[(3,), float32], %bn_data_beta: Tensor[(3,), float32], %bn_data_moving_mean: Tensor[(3,), float32], %bn_data_moving_var: Tensor[(3,), float32], %conv0_weight: Tensor[(64, 3, 7, 7), float32], %bn0_gamma: Tensor[(64,), float32], %bn0_beta: Tensor[(64,), float32], %bn0_moving_mean: Tensor[(64,), float32], %bn0_moving_var: Tensor[(64,), float32], %stage1_unit1_bn1_gamma: Tensor[(64,), float32], %stage1_unit1_bn1_beta: Tensor[(64,), float32], %stage1_unit1_bn1_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn1_moving_var: Tensor[(64,), float32], %stage1_unit1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_bn2_gamma: Tensor[(64,), float32], %stage1_unit1_bn2_beta: Tensor[(64,), float32], %stage1_unit1_bn2_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn2_moving_var: Tensor[(64,), float32], %stage1_unit1_conv2_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_sc_weight: Tensor[(64, 64, 1, 1)

We can now run the customized pass we defined above to schedule individual convolutions on the GPU.

In [0]:
resnet = schedule_conv2d_on_gpu(resnet)
print(resnet)


v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32], %bn_data_gamma: Tensor[(3,), float32], %bn_data_beta: Tensor[(3,), float32], %bn_data_moving_mean: Tensor[(3,), float32], %bn_data_moving_var: Tensor[(3,), float32], %conv0_weight: Tensor[(64, 3, 7, 7), float32], %bn0_gamma: Tensor[(64,), float32], %bn0_beta: Tensor[(64,), float32], %bn0_moving_mean: Tensor[(64,), float32], %bn0_moving_var: Tensor[(64,), float32], %stage1_unit1_bn1_gamma: Tensor[(64,), float32], %stage1_unit1_bn1_beta: Tensor[(64,), float32], %stage1_unit1_bn1_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn1_moving_var: Tensor[(64,), float32], %stage1_unit1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_bn2_gamma: Tensor[(64,), float32], %stage1_unit1_bn2_beta: Tensor[(64,), float32], %stage1_unit1_bn2_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn2_moving_var: Tensor[(64,), float32], %stage1_unit1_conv2_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_sc_weight: Tensor[(64, 64, 1, 1)

We can later rewrite away the device annotations to insert the copies.

In [0]:
resnet = relay.ir_pass.rewrite_annotated_ops(resnet, 0)
print(resnet)

v0.0.1
fn (%data: Tensor[(1, 3, 224, 224), float32], %bn_data_gamma: Tensor[(3,), float32], %bn_data_beta: Tensor[(3,), float32], %bn_data_moving_mean: Tensor[(3,), float32], %bn_data_moving_var: Tensor[(3,), float32], %conv0_weight: Tensor[(64, 3, 7, 7), float32], %bn0_gamma: Tensor[(64,), float32], %bn0_beta: Tensor[(64,), float32], %bn0_moving_mean: Tensor[(64,), float32], %bn0_moving_var: Tensor[(64,), float32], %stage1_unit1_bn1_gamma: Tensor[(64,), float32], %stage1_unit1_bn1_beta: Tensor[(64,), float32], %stage1_unit1_bn1_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn1_moving_var: Tensor[(64,), float32], %stage1_unit1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_bn2_gamma: Tensor[(64,), float32], %stage1_unit1_bn2_beta: Tensor[(64,), float32], %stage1_unit1_bn2_moving_mean: Tensor[(64,), float32], %stage1_unit1_bn2_moving_var: Tensor[(64,), float32], %stage1_unit1_conv2_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_sc_weight: Tensor[(64, 64, 1, 1)

Finally we will look at a couple case studies of what can be built using Relay. We will first look at how Relay is used as a backend in PyTorch integration, then how Relay can be used to compile a model down to traditional hardware, and finally how it can be used to support a custom accelerator, VTA, which we will dicuss in detail today.

## Ahead of time compilation

An example of what can be built using Relay can be found with the [ahead of time compiler](https://github.com/uwsampl/relay-aot). 

We have already installed the compiler above, we can simply import it and use it below.

In [0]:
mlp, params = relay.testing.mlp.get_workload(1)
cmlp = aot.compile(mlp, relay.Module({}), ctx=tvm.cpu(0), tgt=tvm.target.create('llvm'))
data = numpy.ones((1, 1, 28, 28)).astype('float32')
print(cmlp(data, *params.values()))

[[0.08121707 0.44024524 0.02694191 0.03999249 0.01791077 0.21629152
  0.01342565 0.00279213 0.12026625 0.04091699]]


## PyTorch Integration

Recently Facebook engineers have begun to integrate TVM into PyTorch. The are using PyTorch's JIT functionality to generate Relay code which is then optimized and deployed. This support is currently being integrated into mainline TVM, and installed from [here](https://github.com/pytorch/tvm). 

Initial results are promising and a recent writeup on the design can be found on our [blog](https://tvm.ai/2019/05/30/pytorch-frontend). 

![results](https://i.imgur.com/KfJ7oas.png)


```python
def add(a, b, c):
    return a + b + c

# via tracing
relay_graph = torch_tvm.to_relay(add, inputs)

@torch.jit.script
def mul(a, b, c):
    return a * b * c

# via script
relay_graph = torch_tvm.to_relay(mul, inputs)
```